Progress Report:
- DONE: Feb 14: can output a file written with comments but error in delimiters, every character delimited by ','
    Also, Python error about being unable to transcribe unicode for a certain string about 3 secs into loop in get_comments()

In [33]:
import json
import pandas as pd
import requests
import numpy as np
import pandas as pd
import csv
import datetime
import re

### get_reddit(): returns .json object

In [26]:
def get_reddit(subreddit,listing,limit,timeframe):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()

In [25]:
# set parameters for request

subreddit = 'asianamerican'
limit = 100
timeframe = 'all' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top

In [27]:
r = get_reddit(subreddit, listing, limit, timeframe)

In [28]:
def get_post_titles(r):
    '''
    Get a List of post titles
    '''
    posts = []
    for post in r['data']['children']:
        x = post['data']['title']
        posts.append(x)
    return posts

In [29]:
titles = get_post_titles(r)

In [30]:
print(len(titles))

100


In [31]:
print(titles[0:5])

['NBA All-Star Damian Lillard wears ""Stop Asian Hate" shirt while sitting out tonight\'s game', '“Fuck you! We will stop the Hate!” NBA star Baron Davis stands against anti-Asian racism.', 'Accurate', "My friend's mother was one of the victims of Anti-Asian shooting that recently occurred. The gofundme already lays out their situation but she was a single mother working hard to support her kids through school and the pandemic. Any contributions would mean the world", 'Megan Thee Stallion donates $50K to AAPI legal advocacy group after Atlanta spa shootings']


In [9]:
# print keys/elements of a Reddit object (in this case, submission (t3))
list_red_objs = list(search['data']['children'][0]['data'].keys())

# important keys: (1) id (Reddit thing ID), (2) title, (3) ups (upvotes), 
# (4) downs (downvotes), (5) score (not sure), (6) likes (not sure), (7) num_comments)

NameError: name 'search' is not defined

In [22]:
# What categories do I need from each post?
## Comments
## 

#print(r['data']['children'])
dict_ = pd.DataFrame.from_dict(r['data']['children'][0])
dict_

NameError: name 'r' is not defined

### get_search()
- gets the .json file of a Reddit search

In [2]:
# Corrected request url
#https://www.reddit.com/r/asianamerican/search/?q=affirmative%20action&restrict_sr=1&sort=top

def get_search(subreddit,query,limit,timeframe,sort,restrict_sr):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/search.json?q={query}&restrict_sr={restrict_sr}&limit={limit}&t={timeframe}&sort={sort}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()

### Get top 10 post (submission) IDs from Search

In [3]:
def get_top_n_submission_info(r, n):
    '''
    Get a List of post titles
    r: .json object
    n: num of top posts
    # important keys: (1) id (Reddit thing ID), (2) title, (3) ups (upvotes), 
    # (4) downs (downvotes), (5) score (not sure), (6) likes (not sure), (7) num_comments)

    Returns: pandas Dataframe object
    '''
    # create 3 empty arrays of length n
    id_arr = np.empty(n, dtype="S10")
    up_arr = np.empty(n)
    down_arr = np.empty(n)
    score_arr = np.empty(n)
    likes_arr = np.empty(n)
    num_com_arr = np.empty(n)
    
    for i, post in enumerate(r['data']['children']):
        id_arr[i] = post['data']['id']
        up_arr[i] = post['data']['ups']
        down_arr[i] = post['data']['downs']
        score_arr[i] = post['data']['score']
        likes_arr[i] = post['data']['likes']
        num_com_arr[i] = post['data']['num_comments']

    # create Dataframe
    df = pd.DataFrame({'id': id_arr, 'upvotes': up_arr, 'downvotes': down_arr, 'score': score_arr, 'likes': likes_arr, 'num_comments': num_com_arr})


    return df

#### Driver file

In [4]:
# get_search() params
sr = 'asianamerican'
q = 'affirmative action'
limit = 100
timeframe = 'all'
sort = 'top'
restrict_sr = 1

# variable for array of titles
titles = None

search = get_search(subreddit=sr,query=q, limit=limit, timeframe=timeframe, sort=sort, restrict_sr=restrict_sr)
df = get_top_n_submission_info(search, limit)

In [5]:
df.head(50)

,id,upvotes,downvotes,score,likes,num_comments
0,b'14m8mf4',241.0,0.0,241.0,NaN,357.0
1,b'hqic5b',223.0,0.0,223.0,NaN,114.0
2,b'9nk0do',212.0,0.0,212.0,NaN,44.0
3,b'sejv5m',152.0,0.0,152.0,NaN,85.0
4,b'h9tuc2',142.0,0.0,142.0,NaN,108.0
5,b'sbrca3',130.0,0.0,130.0,NaN,224.0
6,b'cwk0vc',133.0,0.0,133.0,NaN,134.0
7,b'8ajkwc',122.0,0.0,122.0,NaN,32.0
8,b'8usex7',120.0,0.0,120.0,NaN,42.0
9,b'rjy8ut',108.0,0.0,108.0,NaN,39.0


In [82]:
# get column of t3 object submission IDs
id_arr = df_no_chinese['id'].to_numpy()

# decode strings from byte 
def decode_arr(x):
    return x.decode('utf-8')

vfunc = np.vectorize(decode_arr)
id_arr_decoded = vfunc(id_arr)

print(id_arr_decoded)
print(len(id_arr_decoded))

['14m8mf4' '9nk0do' 'sejv5m' 'h9tuc2' 'sbrca3' 'cwk0vc' '8ajkwc' '8usex7'
 'rjy8ut' '7481yg' '5btr2j' '87ru7q' 'pvb846' 'jny9p4' '3oymmf' '6rolbg'
 '8uonyt' 'sz1qyo' '9meo0b' '8vt7kt' '8rlr3y' 'j7ybkv' '9mzffm' 'jm4x1b'
 'b9rf6b' '3w7268' '6ruvgx' '9wwgwt' '9ozm49' '967i7r' '9z3ssz' '6t02ts'
 '6r0tss' '5cudq1' '3dmbpz' '6rjw6f' 'sbvysg' '62q795' '9s7sz1' '36jedc'
 '3e4d2v' '4kq3uy' '5i6idf' '9kgf7i' '21a4zz' '9oq200' '6riosv' '3euiwo'
 '23wexw' '31whah' '20ewl2' '9rd4tx' '23ouhl' 'anfd61' 'dskb09' '6ra6ey'
 'jka8ii' '105fx43' '9oovpa' '2n01k4' '1v63z3' '8a1j3p' 'h107c4' '9ap203'
 '2rdduz' '1juzmv' '6sqhhh' '3bkavj' '3rgzi3' '12e8uno' 'sujd6e' 'drhgp6'
 '43eu4h' 'mauapi' 'tuqh5o' '88rs2g' '266qsv' '14v43k' '3ija6r' '4eb7qj'
 '2c8iu1' '3rkhqd' '4lqzqq' '4q20ib' '24hmx8' 'esbwk7' 'd6767e' '6ek03l'
 '5cw7uu' '789gct' '1belws6' 'bgj9bw' '4ziaf0']
93


Use id_arr to get comments of these submission id's.

## Get comments

### Using Reddit API

In [23]:
'''
# Don't use below code - use PRAW instead
def get_comments(sr, post_id, context, limit, showedits, showmedia, showmore, showtitle, theme, threaded, truncate, sort):
    try:
        base_url = f'https://www.reddit.com/r/{sr}/comments/{post_id}?context={context}&limit={limit}&showedits={showedits}&showmore={showmore}&showmedia={showmedia}&showtitle={showtitle}&sort={sort}&theme={theme}&threaded={threaded}&truncate={truncate}'
        request = requests.get(base_url, headers = {'User_agent': 'yourbot'})
        return request.json()
    except:
        print('An Error Occured')
'''

In [21]:
'''
sr = 'asianamerican'
context = 5 # int: 0-8
limit = 100
showedits = False
showmore = False
showmedia = False
showtitle = False
sort = 'confidence' # one of (confidence, top, new, controversial, old, random, qa, live)
theme = 'default'
threaded = False
truncate = 25
post_id = 'sejv5m'
'''

In [24]:

'''comments = get_comments(sr=sr,
                        post_id=post_id,
                        context=context,
                        limit=limit,
                        showedits=showedits,
                        showmore=showmore,
                        showtitle=showtitle,
                        showmedia=showmedia,
                        sort=sort,
                        theme=theme,
                        threaded=threaded,
                        truncate=truncate)'''

An Error Occured


### Using PRAW Instead of Reddit API

In [7]:
import praw

# need to take secret and password off of Github in the future

reddit = praw.Reddit(
    client_id="iaKUGuo77BwJYHElyAo1Jg",
    client_secret="19bSH4KJpPJB2WFeYhUe--jk4IKeHQ",
    password='Ehdwn1928K!',
    user_agent="Comment Extraction u/dlehdwn",
    username='dlehdwn',
)

In [77]:
print(reddit.user.me())

dlehdwn


In [71]:
#enable read-only mode
#reddit.read_only = True

In [6]:
def get_comments(praw_obj, id_arr, filename):
    '''
    praw_obj: PRAW Reddit object
    id_arr: array of Reddit t3 IDs
    '''
    #open a file to write to
    with open(filename, 'w', newline='', encoding="utf-8") as f:

        # writer
        writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        # for each submission/post
        for id in id_arr:
            sub = praw_obj.submission(id)
            
            # get post data
            time = datetime.datetime.fromtimestamp(sub.created)
            post_text = sub.title + ' ' + sub.selftext
            parent_id = None

            # submission author can be None
            if sub.author is not None:
                username = sub.author.name
                flair = sub.author_flair_text
                writer.writerow([id, parent_id, username, time, flair, post_text])

            else:
                #print('comment.author is None')
                #print(f'body: {body}\n')
                writer.writerow([id, parent_id, None, time, None, post_text])

            # get all comments
            sub.comments.replace_more(limit=None, threshold=1) #threshold - min num of comments needed to traverse into "More comments" object
            for comment in sub.comments.list():

                # get comment features
                comment_time = datetime.datetime.fromtimestamp(comment.created)
                comment_id = comment.id
                comment_parent_id = comment.parent_id
                body = comment.body

                # get username of comment author
                if comment.author is not None:
                    comment_username = comment.author.name
                    comment_flair = comment.author_flair_text
                    writer.writerow([comment_id, comment_parent_id, comment_username, comment_time, comment_flair, body])
                
                else:
                    #print('comment.author is None')
                    #print(f'body: {body}\n')
                    writer.writerow([comment_id, comment_parent_id, None, comment_time, None, body])

    f.close()

In [84]:
num_comments_collected = get_comments(reddit, id_arr_decoded, 'top_100_post_comments_no_chinese.txt') #takes a min to run

In [85]:
# num of actual total comments
print(df['num_comments'].sum())

# num of collected comments
print(num_comments_collected)

3707.0
None


#### Num of comments collected
- Num of collected posts and comments in top 100 posts: 3633
- Num of actual total posts and comments in top 100 posts: 3709

### Example with one submission

In [8]:
submission = reddit.submission("14m8mf4")

In [9]:
import datetime

# get datetime of post
created = submission.created
time = datetime.datetime.fromtimestamp(created)
print(time)

2023-06-29 10:54:44


In [10]:
if submission.author is not None:
    username = submission.author.name
    flair = submission.author_flair_text
    time = datetime.datetime.fromtimestamp(submission.created)
    time_utc = datetime.datetime.fromtimestamp(submission.created_utc) # created and created_utc return the same time
    self_text = submission.selftext
    title = submission.title
    id = submission.id

print(id)
print(title)
print(username)
print(flair)
print(time)
print(time_utc)
print(self_text)

14m8mf4
[Megathread] Supreme Court Ruling on Affirmative Action
Tungsten_
None
2023-06-29 10:54:44
2023-06-29 10:54:44
This is a consolidated thread for users to discuss today's supreme court decision on affirmative action at Harvard and UNC. Please, even in disagreement, be civil and kind.

[NBC](https://www.nbcnews.com/politics/supreme-court/supreme-court-strikes-affirmative-action-programs-harvard-unc-rcna66770)

[CNN](https://www.cnn.com/politics/live-news/supreme-court-decisions/index.html)

[NYT](https://www.nytimes.com/live/2023/06/29/us/affirmative-action-supreme-court)

[WaPo](https://www.washingtonpost.com/education/2023/06/29/supreme-court-student-loan-forgiveness-affirmative-action/)

[Supreme Court Opinion](https://www.supremecourt.gov/opinions/22pdf/20-1199_hgdj.pdf)


In [35]:
def user_comments_posts_exclude_sr(username, regex, sr='t5_2rfyw'):

    submissions_sublist = reddit.redditor(username).submissions.new(limit=None)
    subm_count = 0

    # get submissions of this user
    for subm in submissions_sublist:
        if subm.subreddit_id != 't5_2rfyw':
            if regex.search(subm.title.lower()):
                print(f'title:{subm.title}')

            if regex.search(subm.selftext.lower()):
                print(f'selftext:{subm.selftext}')
                
            subm_count+=1

    if subm_count == 100:
        print('Submission count 100')


    comment_count = 0

    # get comments of this user from other subreddits
    comments_sublist = reddit.redditor(username).comments.new(limit=None)
    for comment in comments_sublist:
        if comment.subreddit_id != 't5_2rfyw':
            #print(comment.body)
            if regex.search(comment.body.lower()):
                print(f'comment:{comment.body}')
        comment_count+=1
    if comment_count == 100:
        print('Comment count 100')


In [32]:
comments_df = pd.read_csv('C:/Users/danie/Study/MACSS/macs30200/scraper/top_100_post_comments_user_time_text.txt', header=None, names=['id', 'parent_id','username', 'time_created', 'flair', 'body'])

In [41]:
regex = re.compile(r'affirma?t?i?v?e?\saction')
usernames = comments_df['username'].unique() # get rid of nan
vfunc = np.vectorize(lambda x: user_comments_posts_exclude_sr(x, regex=regex))
vfunc(usernames)

comment:Most do not support race-based factors in admissions. 74% have heard of affirmative action, 53% support affirmative action, **23% believe colleges should consider race and ethnicity when deciding which students to accept.** 

https://www.pewresearch.org/race-ethnicity/2023/06/08/asian-americans-hold-mixed-views-around-affirmative-action/

This is the tricky thing about polls, the wording of the question matters, supporting AA generally is not the same as AA in schools. I find it disingenuous that these authors from NBC, Guardian are cherry-picking a narrative that suits them the best rather than painting the whole picture of the issue. It is frankly racist to paint Asians as confused minorities that need white liberals to tell them what is right.
comment:Apparently Affirmative action benefited white women the most. That surprised me. The fight against affirmative action feels moot in the long term, because people who were rejected from colleges will find another reason to chase

TypeError: __str__ returned non-string (type float)

In [6]:
# print top level comments
for top_level_comment in submission.comments:
    print(top_level_comment.body)

Thanks to everyone who engaged in insightful and respectful discourse about the news. 

This thread is now locked for comments.
I would prefer using a process that takes into account poverty instead. 

The first generation of my family that came to America was *painfully* poor and everyone showed up with neither money nor education. They worked in kitchens and laundromats. Notice a lot of people in bigger reddit boards talking shit about the "Chinese billionaire" boogeyman (fearmongering like this also erases the less visible Asian races who came to America as refugees and reduces all Asians to a monolithic "rich Asian stereotype") and how this will only help them. The Chinese people I know were not coming to America with bags of cash.
u/Tungsten_, Thanks for creating a section just to discuss this. When I read the news I immediately went searching for a forum where folks might have civil discourse on this topic.

Just had a few comments/questions:

1. Has anyone come across seemingly 

In [10]:
submission.comments.replace_more(limit=None)
for comment in submission.comments.list()[:10]:
    #print(comment.body)
    #print(datetime.datetime.fromtimestamp(comment.created))
    print(comment.parent_id)
    

t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4
t3_14m8mf4


## Find posts/submissions that mention Chinese Americans

In [6]:
# Corrected request url
#https://www.reddit.com/r/asianamerican/search/?q=affirmative%20action&restrict_sr=1&sort=top

def get_search(subreddit,query,limit,timeframe,sort,restrict_sr):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/search.json?q={query}&restrict_sr={restrict_sr}&limit={limit}&t={timeframe}&sort={sort}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()

In [35]:
def get_top_n_submission_info(r, n):
    '''
    Get a List of post titles
    r: .json object
    n: num of top posts
    # important keys: (1) id (Reddit thing ID), (2) title, (3) ups (upvotes), 
    # (4) downs (downvotes), (5) score (not sure), (6) likes (not sure), (7) num_comments)

    Returns: pandas Dataframe object
    '''
    # create 3 empty arrays of length n
    id_arr = np.empty(n, dtype="S10")
    #up_arr = np.empty(n)
    #down_arr = np.empty(n)
    score_arr = np.empty(n)
    #likes_arr = np.empty(n)
    num_com_arr = np.empty(n)
    title_arr = np.empty(n, dtype='object')
    self_text_arr = np.empty(n, dtype='object')

    
    for i, post in enumerate(r['data']['children']):
        id_arr[i] = post['data']['id']
        #up_arr[i] = post['data']['ups']
        #down_arr[i] = post['data']['downs']
        score_arr[i] = post['data']['score']
        #likes_arr[i] = post['data']['likes']
        title_arr[i] = post['data']['title']
        num_com_arr[i] = post['data']['num_comments']
        self_text_arr[i] = post['data']['selftext']

    # create Dataframe
    df = pd.DataFrame({'id': id_arr, 'score': score_arr, 'title': title_arr, 'self_text': self_text_arr, 'num_comments': num_com_arr})


    return df

In [36]:
# get_search() params
sr = 'asianamerican'
q = 'affirmative action'
limit = 100
timeframe = 'all'
sort = 'top'
restrict_sr = 1

# variable for array of titles
titles = None

search = get_search(subreddit=sr,query=q, limit=limit, timeframe=timeframe, sort=sort, restrict_sr=restrict_sr)
df = get_top_n_submission_info(search, limit)

In [57]:
chinese_str = ['chinese', 'chinese-americans', 'chinese-american','chineseamerican', 'chineseamericans']

chinese_rows = df[df.title.str.lower().str.contains('chinese') | df.self_text.str.lower().str.contains('chinese')]

In [58]:
chinese_rows

,id,score,title,self_text,num_comments
1,b'hqic5b',225.0,A big gap in support for affirmative action ha...,,114.0
26,b'ti11r7',32.0,Asian Countries which represent the ancestral ...,I just googled search for the first time affir...,66.0
35,b'5cudq1',25.0,We Will Not Be Used: Are Asian-Americans the R...,"By Mari J. Matsuda\n\nAmerican lawyer, activis...",6.0
45,b'dezvi6',21.0,Opinion | Why have Chinese immigrants become t...,,25.0
46,b'9kgf7i',20.0,2018 r/AsianAmerican Survey Results,## BASIC DEMOGRAPHICS:\n\n* [Age](https://i.im...,40.0
55,b'1yujng',15.0,Should AAs (Asian-Americans) support AA (Affir...,I work at a mostly Chinese-American company in...,177.0
58,b'hlnwtr',17.0,"Lawmakers Ting, Chiu hearing from Chinese Amer...",,2.0
60,b'ldzz0f',12.0,Affirmative action and rise of the Chinese-Ame...,,15.0
63,b'105fx43',14.0,need advice and insights on traditional mother...,hi all! I have been in this r/ for a while and...,20.0
69,b'1juzmv',11.0,Affirmative Action – Harming Asians Most,I hate how Asians are stereotyped as being “su...,26.0


In [59]:
print(chinese_rows.shape)

(11, 5)


Of these 11 rows that mention Chinese Americans in the title or selftext, how many of them mention other Asian ethnicities?

In [65]:
regex_other_asian = r'((koreans?)|(japanese)|(indians?)|(filipinos?)|(vietnamese))'

In [66]:
other_asian = chinese_rows[chinese_rows.title.str.lower().str.contains(regex_other_asian) | df.self_text.str.lower().str.contains(regex_other_asian)]

c:\Users\danie\anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
c:\Users\danie\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [67]:
other_asian

,id,score,title,self_text,num_comments
35,b'5cudq1',25.0,We Will Not Be Used: Are Asian-Americans the R...,"By Mari J. Matsuda\n\nAmerican lawyer, activis...",6.0
46,b'9kgf7i',20.0,2018 r/AsianAmerican Survey Results,## BASIC DEMOGRAPHICS:\n\n* [Age](https://i.im...,40.0
63,b'105fx43',14.0,need advice and insights on traditional mother...,hi all! I have been in this r/ for a while and...,20.0
69,b'1juzmv',11.0,Affirmative Action – Harming Asians Most,I hate how Asians are stereotyped as being “su...,26.0


In [68]:
other_asian_ids = other_asian['id']

In [70]:
chinese_rows_final = chinese_rows[~chinese_rows['id'].isin(other_asian_ids)]

In [74]:
df_no_chinese = df[~df['id'].isin(chinese_rows_final['id'])]

In [75]:
df_no_chinese

,id,score,title,self_text,num_comments
0,b'14m8mf4',244.0,[Megathread] Supreme Court Ruling on Affirmati...,This is a consolidated thread for users to dis...,357.0
2,b'9nk0do',210.0,"Anti-Asian Bias, Not Affirmative Action, Is on...",,44.0
3,b'sejv5m',148.0,Opinion | It’s Time for an Honest Conversation...,,85.0
4,b'h9tuc2',139.0,University of California regents unanimously e...,,108.0
5,b'sbrca3',134.0,Supreme Court to hear Harvard affirmative acti...,[https://www.cnn.com/2022/01/24/politics/supre...,224.0
...,...,...,...,...,...
95,b'5cw7uu',5.0,LGBT Culture has poor knowledge on Indians and...,I need to state the following points about the...,11.0
96,b'789gct',4.0,Assault on Affirmative Action Unwarranted [By ...,,1.0
97,b'1belws6',4.0,How White Supremacy uses Taiwan,This is a long read but I seriously feel the n...,1.0
98,b'bgj9bw',5.0,Does this sub really have a disdain for Affirm...,After looking up some information about Affirm...,19.0
